In [1]:
# skorch : pytorch를 쉽게 구현할 수 있는 라이브러리
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [2]:
X_train = X_train.reshape(-1,784)
X_test = X_test.reshape(-1,784)
X_train = X_train / 255.
X_test = X_test / 255.

In [3]:
import torch
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train.astype('int32')).long()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test.astype('int32')).long()
print(X_train.shape)
print(X_test.shape)
X_train=X_train.cuda()
y_train=y_train.cuda()
X_test=X_test.cuda()
y_test=y_test.cuda()

torch.Size([60000, 784])
torch.Size([10000, 784])


In [4]:
import torch.nn as nn
import torch.nn.functional as F

# 신경망 구성
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=-1)
model=Net().cuda()

In [5]:
# pip install skorch
from skorch import NeuralNetClassifier
net = NeuralNetClassifier(Net,max_epochs=20,lr=0.1)
net.fit(X_train, y_train)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6491       0.8987        0.3506  2.0175
      2        0.3159       0.9181        0.2859  1.8959
      3        0.2647       0.9286        0.2473  2.0151
      4        0.2295       0.9364        0.2194  2.1958
      5        0.2026       0.9423        0.1980  3.3811
      6        0.1811       0.9481        0.1816  2.1629
      7        0.1632       0.9526        0.1681  2.0698
      8        0.1481       0.9559        0.1571  2.1118
      9        0.1352       0.9582        0.1476  3.4345
     10        0.1241       0.9590        0.1400  1.8541
     11        0.1146       0.9617        0.1332  1.7332
     12        0.1063       0.9634        0.1275  1.8909
     13        0.0990       0.9642        0.1224  1.8642
     14        0.0926       0.9661        0.1182  1.9005
     15        0.0869       0.9668        0.1142  1.9001
     16        0.0817       0.9

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Net(
    (fc1): Linear(in_features=784, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=10, bias=True)
  ),
)

In [6]:
import numpy as np
# 모형의 정확도 계산
pred = net.predict(X_test)
y_test = y_test.cpu().numpy()
accuracy = np.mean(pred == y_test)
accuracy

0.9724

In [7]:
from torchinfo import summary
summary(model, input_size=(100,784))

Layer (type:depth-idx)                   Output Shape              Param #
Net                                      [100, 10]                 --
├─Linear: 1-1                            [100, 128]                100,480
├─Linear: 1-2                            [100, 10]                 1,290
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
Total mult-adds (M): 10.18
Input size (MB): 0.31
Forward/backward pass size (MB): 0.11
Params size (MB): 0.41
Estimated Total Size (MB): 0.83